In [24]:
import pandas as pd
from glob import glob


In [25]:
infos = glob('./info*')
df = pd.DataFrame(infos, columns = ['info_path'])
df['meta'] = df.info_path.apply(lambda x: pd.read_csv(x, index_col=0)['0'].to_dict())
df.meta.apply(pd.Series).sort_values('dataset')

,dataset,y,x,MEGMA,key,X_size,fmap_shape
1,CRC,./metagenomics/colphy_y.csv,./metagenomics/colphy_x.csv,"('umap', 60)",MEGMA-CRC-UMAP-10,"(121, 23, 22, 60)","(23, 22)"
4,CRC,./metagenomics/colphy_y.csv,./metagenomics/colphy_x.csv,"('umap', 5)",MEGMA-CRC-UMAP-5,"(121, 23, 22, 5)","(23, 22)"
3,Cirrhosis,./metagenomics/cirphy_y.csv,./metagenomics/cirphy_x.csv,"('umap', 60)",MEGMA-Cirrhosis-UMAP-10,"(232, 24, 23, 60)","(24, 23)"
7,Cirrhosis,./metagenomics/cirphy_y.csv,./metagenomics/cirphy_x.csv,"('umap', 5)",MEGMA-Cirrhosis-UMAP-5,"(232, 24, 23, 5)","(24, 23)"
5,IBD,./metagenomics/ibdphy_y.csv,./metagenomics/ibdphy_x.csv,"('umap', 5)",MEGMA-IBD-UMAP-5,"(110, 22, 21, 5)","(22, 21)"
8,IBD,./metagenomics/ibdphy_y.csv,./metagenomics/ibdphy_x.csv,"('umap', 60)",MEGMA-IBD-UMAP-10,"(110, 22, 21, 60)","(22, 21)"
6,Obesity,./metagenomics/obephy_y.csv,./metagenomics/obephy_x.csv,"('umap', 5)",MEGMA-Obesity-UMAP-5,"(253, 22, 22, 5)","(22, 22)"
9,Obesity,./metagenomics/obephy_y.csv,./metagenomics/obephy_x.csv,"('umap', 60)",MEGMA-Obesity-UMAP-10,"(253, 22, 22, 60)","(22, 22)"
0,T2D,./metagenomics/t2dphy_y.csv,./metagenomics/t2dphy_x.csv,"('umap', 5)",MEGMA-T2D-UMAP-5,"(344, 24, 24, 5)","(24, 24)"
2,T2D,./metagenomics/t2dphy_y.csv,./metagenomics/t2dphy_x.csv,"('umap', 60)",MEGMA-T2D-UMAP-10,"(344, 24, 24, 60)","(24, 24)"


In [26]:
def _get_mean(x):
    l1 = pd.read_csv(x, index_col=0)
    return pd.DataFrame(l1).groupby('repeat_seed').mean().mean()

def _get_std(x):
    l1 = pd.read_csv(x, index_col=0)
    return pd.DataFrame(l1).groupby('repeat_seed').mean().std()


pfs = glob('./MEGMA*')
df = pd.DataFrame(pfs, columns = ['result_path'])


mean = df.result_path.apply(lambda x: _get_mean(x))
std = df.result_path.apply(lambda x: _get_std(x))

df = df.result_path.apply(lambda x:x.split('-')).apply(pd.Series)
df.columns = ['Method', 'dataset', 'embd', 'channel']


mean_stds = []
for x in mean.columns[:-1]:
    s = mean[x].round(3).astype(str) + '±' + std[x].round(3).astype(str)
    s.to_frame(name = x)
    mean_stds.append(s)
    
mean_stds = pd.concat(mean_stds, axis=1)

df = df.join(mean_stds)
df = df.sort_values('dataset')

In [27]:
df

,Method,dataset,embd,channel,acc,auc,mcc,pres,recall,f1
4,./MEGMA,CRC,UMAP,10.csv,0.717±0.017,0.826±0.012,0.39±0.042,0.728±0.028,0.717±0.017,0.676±0.023
7,./MEGMA,CRC,UMAP,5.csv,0.709±0.018,0.808±0.014,0.381±0.046,0.728±0.029,0.709±0.018,0.669±0.024
3,./MEGMA,Cirrhosis,UMAP,5.csv,0.868±0.004,0.941±0.003,0.743±0.009,0.875±0.005,0.868±0.004,0.868±0.005
9,./MEGMA,Cirrhosis,UMAP,10.csv,0.884±0.011,0.95±0.004,0.777±0.022,0.893±0.01,0.884±0.011,0.883±0.011
0,./MEGMA,IBD,UMAP,5.csv,0.788±0.006,0.858±0.029,0.122±0.027,0.661±0.012,0.788±0.006,0.708±0.008
5,./MEGMA,IBD,UMAP,10.csv,0.779±0.009,0.865±0.027,0.044±0.059,0.618±0.026,0.779±0.009,0.685±0.016
2,./MEGMA,Obesity,UMAP,5.csv,0.647±0.008,0.655±0.016,0.034±0.038,0.502±0.038,0.647±0.008,0.531±0.012
6,./MEGMA,Obesity,UMAP,10.csv,0.653±0.014,0.671±0.018,0.174±0.027,0.632±0.012,0.653±0.014,0.62±0.012
1,./MEGMA,T2D,UMAP,10.csv,0.666±0.012,0.726±0.007,0.335±0.025,0.67±0.013,0.666±0.012,0.663±0.013
8,./MEGMA,T2D,UMAP,5.csv,0.672±0.011,0.728±0.009,0.349±0.021,0.677±0.01,0.672±0.011,0.67±0.011
